# Atmospherically Corrected Earth Engine Time Series 

### Overview

This notebook creates atmospherically corrected time series of satellite imagery using [Google Earth Engine](https://earthengine.google.com/) and the [6S emulator](https://github.com/samsammurphy/6S_emulator). 

### Supported missions

* Sentintel2
* Landsat8
* Landsat7
* Landsat5
* Landsat4

### Output

Average, cloud-*free* pixel values 

### Cloud masking

Uses standard cloud masks, i.e. FMASK for Landsat and ESA-QA60 for Sentinel 2.  There is no guarantee they will find all clouds, a discussion on more advance and/or alternative cloud masking strategies is available [here](https://groups.google.com/forum/#!searchin/google-earth-engine-developers/cloud$20AND$20sentinel2%7Csort:relevance/google-earth-engine-developers/i63DS-Dg8Sg/FWenONUFBwAJ)

#### Initialize

In [1]:
# standard modules
import os
import sys
import ee
import colorsys
from IPython.display import display, Image
%matplotlib inline
ee.Initialize()

# custom modules
base_dir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(base_dir,'atmcorr'))
from kml_reader import read_kml
from timeSeries import timeSeries
from postProcessing import postProcessing
from plots import plotTimeSeries

ImportError: cannot import name 'plotTimeSeries'

In [ ]:
#debuggin
%load_ext autoreload
%autoreload 2

### User Input

In [ ]:
target = 'Yugama'
geom = read_kml('IAVCEI.kml', target)

# start and end of time series
startDate = '1990-01-01'# YYYY-MM-DD
stopDate  = '2017-01-01'# YYYY-MM-DD

# satellite missions
missions = ['Sentinel2', 'Landsat8', 'Landsat7', 'Landsat5', 'Landsat4']

### All time series
This function extracts cloud-free time series for each mission, atmospherically corrects them and joins them together

In [ ]:
allTimeSeries = timeSeries(target, geom, startDate, stopDate, missions)

### Data post-processing
Resample into daily intervals using liner interpolation and calculate hue-saturation-value from RGB.

In [ ]:
DF = postProcessing(allTimeSeries, startDate, stopDate)

### Hue Stretch
We visualize *hue* by taking a [HSV](http://infohost.nmt.edu/tcc/help/pubs/colortheory/web/hsv.html) color triplet and 'strecthing' the *saturation*  and *value* (i.e. setting them to 1) then converting the new 'stretched' HSV color-triplet back into RGB for display on the screen.

In [ ]:
hue_stretch = [colorsys.hsv_to_rgb(hue,1,1) for hue in DF['hue']]

### bringing it all together...
make a pretty graph to help us do some science.

In [ ]:
plotTimeSeries(DF, hue_stretch, startDate, stopDate)

Thue hue looks kinda like tree rings through time..

..may be useful for phenology and/or wavelet analysis?